In [4]:
%%bash

CUDA_VISIBLE_DEVICES=0 python scPheno.py --sup-data-file "/home/zengbio/Project/scPheno_Ex/ifnb_tutorial/ifnb.txt" \
                        --sup-label-file "/home/zengbio/Project/scPheno_Ex/ifnb_tutorial/ifnb_celltype.txt" \
                        --sup-condition-file "/home/zengbio/Project/scPheno_Ex/ifnb_tutorial/ifnb_cellsubtype.txt" \
                        --sup-condition2-file "/home/zengbio/Project/scPheno_Ex/ifnb_tutorial/ifnb_condition.txt" \
                        -lr 0.0001 \
                        -n 200 \
                        -bs 100 \
                        --cuda \
                        -zi \
                        -likeli negbinomial \
                        -dirichlet \
                        -cv 0 \
                        --label-type categorical \
                        --condition-type categorical \
                        --condition2-type categorical \
                        --save-model ifnb.pth

1 epoch: avg losses 523891.6277 0.0000 elapsed 6.5435 seconds
2 epoch: avg losses 371362.4982 0.0000 elapsed 6.0781 seconds
3 epoch: avg losses 333902.0080 0.0000 elapsed 5.9512 seconds
4 epoch: avg losses 315128.9750 0.0000 elapsed 5.8860 seconds
5 epoch: avg losses 308834.1897 0.0000 elapsed 5.9118 seconds
6 epoch: avg losses 305722.5067 0.0000 elapsed 7.6892 seconds
7 epoch: avg losses 302105.4826 0.0000 elapsed 6.2899 seconds
8 epoch: avg losses 297673.5692 0.0000 elapsed 5.9438 seconds
9 epoch: avg losses 294605.5906 0.0000 elapsed 6.5041 seconds
10 epoch: avg losses 291018.8487 0.0000 elapsed 6.6551 seconds
11 epoch: avg losses 287268.1464 0.0000 elapsed 6.1560 seconds
12 epoch: avg losses 284127.7589 0.0000 elapsed 5.9135 seconds
13 epoch: avg losses 281902.7205 0.0000 elapsed 5.8428 seconds
14 epoch: avg losses 279126.2893 0.0000 elapsed 5.6982 seconds
15 epoch: avg losses 275912.8652 0.0000 elapsed 5.6918 seconds
16 epoch: avg losses 273133.6629 0.0000 elapsed 5.6770 seconds
1

In [5]:
import numpy as np
import pandas as pd
import datatable as dt

from scPheno import scPheno
from utils.scdata_cached import SingleCellCached

import torch
from torch.utils.data import DataLoader




In [6]:
ModelPath = 'ifnb.pth'
DataPath='/home/zengbio/Project/scPheno_Ex/ifnb_tutorial/ifnb.txt'
LabelPath=None
ConditionPath=None
ConditionPath2=None


In [7]:
# load model
model = torch.load(ModelPath)

batch_size = 10000

use_float64 = False
use_cuda = True

In [8]:
# load data
data_cached = SingleCellCached(DataPath, LabelPath, ConditionPath, ConditionPath2, 'condition', use_cuda=False, use_float64 = use_float64)
data_loader = DataLoader(data_cached, batch_size = batch_size, shuffle = False)

In [13]:
# predict conditions
exprs = []
# use the appropriate data loader
for xs,ys,ks,ks2 in data_loader:
    # use classification function to compute all predictions for each batch
    if use_cuda:
        xs = xs.cuda()

    expr = model.generate_expression(xs, mute_label=False, mute_condition=True, mute_condition2=True, mute_noise=True)

    if use_cuda:
        expr = expr.cpu().detach().numpy()
    else:
        expr = expr.detach().numpy()

    exprs.append(expr)


exprs = np.concatenate(exprs, axis=0)

exprs = 10 * exprs / exprs.max()


In [14]:
cells = pd.read_csv('/home/zengbio/Project/scPheno_Ex/ifnb_tutorial/ifnb_cells.txt', header=None, index_col=None)
genes = pd.read_csv('/home/zengbio/Project/scPheno_Ex/ifnb_tutorial/ifnb_genes.txt', header=None, index_col=None)
cells.shape

(13999, 1)

In [15]:
df = pd.DataFrame(exprs, columns=genes[0].values, index=cells[0].values)
dt.Frame(df.reset_index()).to_csv('/home/zengbio/Project/scPheno_Ex/ifnb_tutorial/ifnb_celltype_denoised_expression.txt')